In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'dataset-defended-close-world:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4540211%2F7762823%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240331%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240331T055132Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0371dc26f063aae4d7002482cf9c57635ef14b342c13b7e4fb19e503fe62c65b774d8c66f7c1a8d2850271410a1787c60f8ae655b0517337c473eeead3183846d5986bc81a3e9a72396df3fb32a7a6338b881343163d2d57ed532ce250e9da5b0f886cace5b3438e7b88c08107f873cc04326ff6a5f5a68807efbb8f2560bc438156c3971a58240ae3872024a6cbf02914307e8730d531843fa8fdccb281d88adfd6ad639c7cca3f59c787005ea9f9900548594e5770ccb565bf1596cf15e1b6f4f1a22f58f78dbe322903fc110617d9bf1229baf921a533558b3ed40d1291ad6cd0b9d684150e99ed16362be7a82220d190c10658f99aeb46d5bfca1a72c63a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import tensorflow as tf
import pickle
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.layers import Activation, Flatten, Dense, Dropout, ReLU
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from keras import layers, models

def LoadDataWTFPADCW():

    print("Loading non-defended dataset for closed-world scenario")

    # Point to the directory storing data
    dataset_dir = '/kaggle/input/dataset-defended-close-world/'
    # X represents a sequence of traffic directions
    # y represents a sequence of corresponding label (website's label)
    # Debug: Print dataset directory
    print("Dataset directory:", dataset_dir)

    try:
        # Load training data
        with open(dataset_dir + 'X_train_WTFPAD.pkl', 'rb') as handle:
            X_train = np.array(pickle.load(handle, encoding='latin1'))
        print("X_train loaded")

        with open(dataset_dir + 'y_train_WTFPAD.pkl', 'rb') as handle:
            y_train = np.array(pickle.load(handle, encoding='latin1'))
        print("y_train loaded")

        # Load validation data
        with open(dataset_dir + 'X_valid_WTFPAD.pkl', 'rb') as handle:
            X_valid = np.array(pickle.load(handle, encoding='latin1'))
        print("X_valid loaded")

        with open(dataset_dir + 'y_valid_WTFPAD.pkl', 'rb') as handle:
            y_valid = np.array(pickle.load(handle, encoding='latin1'))
        print("y_valid loaded")

        # Load testing data
        with open(dataset_dir + 'X_test_WTFPAD.pkl', 'rb') as handle:
            X_test = np.array(pickle.load(handle, encoding='latin1'))
        print("X_test loaded")

        with open(dataset_dir + 'y_test_WTFPAD.pkl', 'rb') as handle:
            y_test = np.array(pickle.load(handle, encoding='latin1'))
        print("y_test loaded")

        print("Data dimensions:")
        print("X: Training data's shape : ", X_train.shape)
        print("y: Training data's shape : ", y_train.shape)
        print("X: Validation data's shape : ", X_valid.shape)
        print("y: Validation data's shape : ", y_valid.shape)
        print("X: Testing data's shape : ", X_test.shape)
        print("y: Testing data's shape : ", y_test.shape)

        # Merge datasets
        X_all = np.concatenate((X_train, X_valid, X_test), axis=0)
        y_all = np.concatenate((y_train, y_valid, y_test), axis=0)

        print("Merged data dimensions:")
        print("X: Merged data's shape : ", X_all.shape)
        print("y: Merged data's shape : ", y_all.shape)

        # Print features of the merged dataset
        print("Features of the merged dataset:")
        print(X_all)

        # Check if the class distribution is balanced
        unique_classes, class_counts = np.unique(y_all, return_counts=True)
        class_distribution = dict(zip(unique_classes, class_counts))

        print("Class distribution:")
        for class_label, count in class_distribution.items():
            print(f"Class {class_label}: {count} samples")

        # Plot the class distribution
       # plt.bar(class_distribution.keys(), class_distribution.values())
       # plt.xlabel('Class Label')
       # plt.ylabel('Number of Samples')
      #  plt.title('Class Distribution')
      #  plt.show()

        return X_all, y_all

    except Exception as e:
        print("An error occurred:", str(e))
        return None

# Call the function to load and merge data
X_all, y_all = LoadDataWTFPADCW()


2024-03-29 03:59:40.268322: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 03:59:40.268447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 03:59:40.397729: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading non-defended dataset for closed-world scenario
Dataset directory: /kaggle/input/dataset-defended-close-world/
X_train loaded
y_train loaded
X_valid loaded
y_valid loaded
X_test loaded
y_test loaded
Data dimensions:
X: Training data's shape :  (76000, 5000)
y: Training data's shape :  (76000,)
X: Validation data's shape :  (9500, 5000)
y: Validation data's shape :  (9500,)
X: Testing data's shape :  (9500, 5000)
y: Testing data's shape :  (9500,)
Merged data dimensions:
X: Merged data's shape :  (95000, 5000)
y: Merged data's shape :  (95000,)
Features of the merged dataset:
[[ 1. -1.  1. ... -1. -1.  1.]
 [-1.  1. -1. ...  0.  0.  0.]
 [ 1.  1.  1. ...  0.  0.  0.]
 ...
 [-1.  1. -1. ...  0.  0.  0.]
 [-1.  1. -1. ...  0.  0.  0.]
 [ 1.  1. -1. ...  0.  0.  0.]]
Class distribution:
Class 0: 1000 samples
Class 1: 1000 samples
Class 2: 1000 samples
Class 3: 1000 samples
Class 4: 1000 samples
Class 5: 1000 samples
Class 6: 1000 samples
Class 7: 1000 samples
Class 8: 1000 samples
C

#inplace of adam i want to use adamax

In [ ]:
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.initializers import glorot_uniform
from keras.optimizers import Adamax  # Change here
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import numpy as np

class DFNet:
    @staticmethod
    def build(input_shape, classes, dropout_rate1=0.3, dropout_rate2=0.5, dropout_rate_fc=0.7):
        model = Sequential()

        # ... (previous model definition remains unchanged)
        # Block 1
        filter_num = [None,32, 64, 128, 256]
        kernel_size = [None, 8, 8, 8, 8]
        conv_stride_size = [None, 1, 1, 1, 1]
        pool_stride_size = [None, 4, 4, 4, 4]
        pool_size = [None, 8, 8, 8, 8]

        for i in range(1, 4):
            model.add(Conv1D(filters=filter_num[i], kernel_size=kernel_size[i],
                             strides=conv_stride_size[i], padding='same',
                             input_shape=input_shape if i == 1 else (None, input_shape[1]),
                             name=f'block{i}_conv1'))
            model.add(BatchNormalization(name=f'batch_normalization_{i * 2 - 1}'))
            model.add(Activation('relu', name=f'block{i}_adv_act1'))
            model.add(Conv1D(filters=filter_num[i], kernel_size=kernel_size[i],
                             strides=conv_stride_size[i], padding='same',
                             name=f'block{i}_conv2'))
            model.add(BatchNormalization(name=f'batch_normalization_{i * 2}'))
            model.add(Activation('relu', name=f'block{i}_adv_act2'))

            model.add(MaxPooling1D(pool_size=pool_size[i], strides=pool_stride_size[i],
                                   padding='same', name=f'block{i}_pool'))
            model.add(Dropout(0.1, name=f'block{i}_dropout'))

        # ... (rest of the model remains unchanged)

        model.add(Flatten(name='flatten'))

        # Fully connected layers
        for i in range(1, 3):
            model.add(Dense(512, kernel_initializer=glorot_uniform(seed=0), name=f'fc{i}'))
            model.add(BatchNormalization(name=f'batch_normalization_{i + 8}'))
            model.add(Activation('relu', name=f'fc{i}_act'))

            model.add(Dropout(dropout_rate_fc, name=f'fc{i}_dropout'))

        # Output layer
        model.add(Dense(classes, kernel_initializer=glorot_uniform(seed=0), name='fc_final'))
        model.add(Activation('softmax', name="softmax"))

        print("Model built successfully.")
        return model

# Assuming input_shape and num_classes are defined as follows (adjust based on your actual data)
input_shape = (5000, 1)
num_classes = 95

# Experiment with different dropout rates
dropout_rate1 = 0.6
dropout_rate2 = 0.5
dropout_rate_fc = 0.4

# Build the model with the specified dropout rates
model = DFNet.build(input_shape=input_shape, classes=num_classes,
                    dropout_rate1=dropout_rate1, dropout_rate2=dropout_rate2, dropout_rate_fc=dropout_rate_fc)


# Print the model summary
print("Model Summary:")
model.summary()

# Compile the model
optimizer = Adamax(learning_rate=0.002)  # Change here
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Assuming input_shape and num_classes are defined as follows (adjust based on your actual data)
input_shape = (5000, 1)
num_classes = 95

# Experiment with different dropout rates
dropout_rate1 = 0.6
dropout_rate2 = 0.5
dropout_rate_fc = 0.4

# Load and merge data using the function you defined
X_all, y_all =   LoadDataWTFPADCW()
# Define the number of folds
n_splits = 5  # You can adjust this as needed

# Initialize Stratified K-Fold
stratkf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Lists to store results
all_histories = []

# Iterate over folds
for fold, (train_index, test_index) in enumerate(stratkf.split(X_all, y_all)):
    print(f"\nTraining on Fold {fold + 1}/{n_splits}")

    # Split the data
    X_train, X_test = X_all[train_index], X_all[test_index]
    y_train, y_test = y_all[train_index], y_all[test_index]

        # Build the model
    model = DFNet.build(input_shape=input_shape, classes=num_classes,
                        dropout_rate1=dropout_rate1, dropout_rate2=dropout_rate2, dropout_rate_fc=dropout_rate_fc)


    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
     # Define early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    # Train the model with early stopping
    history = model.fit(X_train, y_train, epochs=25, batch_size=64,
                        validation_data=(X_test, y_test), verbose=2, callbacks=[early_stopping])




    # Train the model
    #history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

    # Evaluate the model on the test set
    loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
    print(f"\nFold {fold + 1} - Test Accuracy: {accuracy * 100:.2f}%")

    # Save history for later analysis if needed
    all_histories.append(history)
    # Check unique labels in y_train

    # Update num_classes based on the actual number of unique classes in your dataset
#num_classes = len(np.unique(y_all))
saved_path_keras = '/kaggle/working/DFNet_OpenWorld_WTFPAD.keras'
model.save(saved_path_keras)

print("Trained model saved successfully at:", saved_path_keras)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model built successfully.
Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ block1_conv1 (Conv1D)           │ (None, 5000, 32)       │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 5000, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_adv_act1 (Activation)    │ (None, 5000, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv1D)           │ (None, 5000, 32)       │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 5000, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_adv_act2 (Activation)    │ (None, 5000, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling1D)      │ (None, 1250, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_dropout (Dropout)        │ (None, 1250, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv1D)           │ (None, 1250, 64)       │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 1250, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_adv_act1 (Activation)    │ (None, 1250, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv1D)           │ (None, 1250, 64)       │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 1250, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_adv_act2 (Activation)    │ (None, 1250, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling1D)      │ (None, 313, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_dropout (Dropout)        │ (None, 313, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv1D)           │ (None, 313, 128)       │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 313, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_adv_act1 (Activation)    │ (None, 313, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv1D)           │ (None, 313, 128)       │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 313, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_adv_act2 (Activation)    │ (None, 313, 128)       │             

 Total params: 5,749,791 (21.93 MB)

...

1188/1188 - 92s - 77ms/step - accuracy: 0.6454 - loss: 1.2667 - val_accuracy: 0.7141 - val_loss: 0.9879
Epoch 3/25
1188/1188 - 92s - 77ms/step - accuracy: 0.7187 - loss: 1.0017 - val_accuracy: 0.7947 - val_loss: 0.7303
Epoch 4/25
1188/1188 - 92s - 77ms/step - accuracy: 0.7573 - loss: 0.8505 - val_accuracy: 0.8097 - val_loss: 0.6653
Epoch 5/25
1188/1188 - 91s - 77ms/step - accuracy: 0.7868 - loss: 0.7414 - val_accuracy: 0.8358 - val_loss: 0.5891
Epoch 6/25
1188/1188 - 91s - 77ms/step - accuracy: 0.8077 - loss: 0.6674 - val_accuracy: 0.8341 - val_loss: 0.5852
Epoch 7/25
1188/1188 - 91s - 77ms/step - accuracy: 0.8264 - loss: 0.5981 - val_accuracy: 0.8504 - val_loss: 0.5402
Epoch 8/25
1188/1188 - 91s - 77ms/step - accuracy: 0.8377 - loss: 0.5547 - val_accuracy: 0.8661 - val_loss: 0.4855
Epoch 9/25
1188/1188 - 91s - 77ms/step - accuracy: 0.8509 - loss: 0.5049 - val_accuracy: 0.8529 - val_loss: 0.5274
Epoch 10/25
1188/1188 - 91s - 77ms/step - accuracy: 0.8589 - loss: 0.4700 - val_accuracy: 0

W0000 00:00:1711696793.302442     105 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


594/594 - 4s - 6ms/step - accuracy: 0.8877 - loss: 0.4381


W0000 00:00:1711696796.425589     105 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



Fold 5 - Test Accuracy: 88.77%
Trained model saved successfully at: /kaggle/working/DFNet_OpenWorld_WTFPAD.keras


#using adam

In [ ]:
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.layers import Activation, Flatten, Dense, Dropout, ReLU
from keras.initializers import glorot_uniform
from keras.optimizers import Adam
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from keras import layers, models
import numpy as np
from keras.layers import Activation

class DFNet:
    @staticmethod
    def build(input_shape, classes, dropout_rate1=0.3, dropout_rate2=0.5, dropout_rate_fc=0.7):
        model = Sequential()

        # ... (previous model definition remains unchanged)
        # Block 1
        filter_num = [None,32, 64, 128, 256]
        kernel_size = [None, 8, 8, 8, 8]
        conv_stride_size = [None, 1, 1, 1, 1]
        pool_stride_size = [None, 4, 4, 4, 4]
        pool_size = [None, 8, 8, 8, 8]

        for i in range(1, 4):
            model.add(Conv1D(filters=filter_num[i], kernel_size=kernel_size[i],
                             strides=conv_stride_size[i], padding='same',
                             input_shape=input_shape if i == 1 else (None, input_shape[1]),
                             name=f'block{i}_conv1'))
            model.add(BatchNormalization(name=f'batch_normalization_{i * 2 - 1}'))
            model.add(Activation('relu', name=f'block{i}_adv_act1'))
            model.add(Conv1D(filters=filter_num[i], kernel_size=kernel_size[i],
                             strides=conv_stride_size[i], padding='same',
                             name=f'block{i}_conv2'))
            model.add(BatchNormalization(name=f'batch_normalization_{i * 2}'))
            model.add(Activation('relu', name=f'block{i}_adv_act2'))

            model.add(MaxPooling1D(pool_size=pool_size[i], strides=pool_stride_size[i],
                                   padding='same', name=f'block{i}_pool'))
            model.add(Dropout(0.1, name=f'block{i}_dropout'))

        # ... (rest of the model remains unchanged)

        model.add(Flatten(name='flatten'))

        # Fully connected layers
        for i in range(1, 3):
            model.add(Dense(512, kernel_initializer=glorot_uniform(seed=0), name=f'fc{i}'))
            model.add(BatchNormalization(name=f'batch_normalization_{i + 8}'))
            model.add(Activation('relu', name=f'fc{i}_act'))

            # Experiment with different dropout rates
            model.add(Dropout(dropout_rate_fc, name=f'fc{i}_dropout'))

        # Output layer
        model.add(Dense(classes, kernel_initializer=glorot_uniform(seed=0), name='fc_final'))
        model.add(Activation('softmax', name="softmax"))

        print("Model built successfully.")
        return model
    # Assuming input_shape and num_classes are defined as follows (adjust based on your actual data)
input_shape = (5000, 1)
num_classes = 95

# Experiment with different dropout rates
dropout_rate1 = 0.6
dropout_rate2 = 0.5
dropout_rate_fc = 0.4

# Build the model with the specified dropout rates
model = DFNet.build(input_shape=input_shape, classes=num_classes,
                    dropout_rate1=dropout_rate1, dropout_rate2=dropout_rate2, dropout_rate_fc=dropout_rate_fc)


# Print the model summary
print("Model Summary:")
model.summary()

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Assuming input_shape and num_classes are defined as follows (adjust based on your actual data)
input_shape = (5000, 1)
num_classes = 95

# Experiment with different dropout rates
dropout_rate1 = 0.6
dropout_rate2 = 0.5
dropout_rate_fc = 0.4

# Load and merge data using the function you defined
X_all, y_all =   LoadDataWTFPADCW()
# Define the number of folds
n_splits = 5  # You can adjust this as needed

# Initialize Stratified K-Fold
stratkf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Lists to store results
all_histories = []

# Iterate over folds
for fold, (train_index, test_index) in enumerate(stratkf.split(X_all, y_all)):
    print(f"\nTraining on Fold {fold + 1}/{n_splits}")

    # Split the data
    X_train, X_test = X_all[train_index], X_all[test_index]
    y_train, y_test = y_all[train_index], y_all[test_index]

        # Build the model
    model = DFNet.build(input_shape=input_shape, classes=num_classes,
                        dropout_rate1=dropout_rate1, dropout_rate2=dropout_rate2, dropout_rate_fc=dropout_rate_fc)


    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
     # Define early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    # Train the model with early stopping
    history = model.fit(X_train, y_train, epochs=20, batch_size=64,
                        validation_data=(X_test, y_test), verbose=2, callbacks=[early_stopping])




    # Train the model
    #history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

    # Evaluate the model on the test set
    loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
    print(f"\nFold {fold + 1} - Test Accuracy: {accuracy * 100:.2f}%")

    # Save history for later analysis if needed
    all_histories.append(history)
    # Check unique labels in y_train

    # Update num_classes based on the actual number of unique classes in your dataset
#num_classes = len(np.unique(y_all))
saved_path_keras = '/kaggle/working/DFNet_OpenWorld_WTFPAD.keras'
model.save(saved_path_keras)

print("Trained model saved successfully at:", saved_path_keras)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model built successfully.
Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ block1_conv1 (Conv1D)           │ (None, 5000, 32)       │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 5000, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_adv_act1 (Activation)    │ (None, 5000, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv1D)           │ (None, 5000, 32)       │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 5000, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_adv_act2 (Activation)    │ (None, 5000, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling1D)      │ (None, 1250, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_dropout (Dropout)        │ (None, 1250, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv1D)           │ (None, 1250, 64)       │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 1250, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_adv_act1 (Activation)    │ (None, 1250, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv1D)           │ (None, 1250, 64)       │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 1250, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_adv_act2 (Activation)    │ (None, 1250, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling1D)      │ (None, 313, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_dropout (Dropout)        │ (None, 313, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv1D)           │ (None, 313, 128)       │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 313, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_adv_act1 (Activation)    │ (None, 313, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv1D)           │ (None, 313, 128)       │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 313, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_adv_act2 (Activation)    │ (None, 313, 128)       │             

 Total params: 5,749,791 (21.93 MB)

...

1188/1188 - 93s - 78ms/step - accuracy: 0.6442 - loss: 1.2735 - val_accuracy: 0.6412 - val_loss: 1.2824
Epoch 3/20
1188/1188 - 142s - 120ms/step - accuracy: 0.7148 - loss: 1.0045 - val_accuracy: 0.7863 - val_loss: 0.7449
Epoch 4/20
1188/1188 - 93s - 78ms/step - accuracy: 0.7576 - loss: 0.8502 - val_accuracy: 0.8227 - val_loss: 0.6319
Epoch 5/20
1188/1188 - 93s - 78ms/step - accuracy: 0.7876 - loss: 0.7453 - val_accuracy: 0.8289 - val_loss: 0.6116
Epoch 6/20
1188/1188 - 93s - 78ms/step - accuracy: 0.8089 - loss: 0.6622 - val_accuracy: 0.8298 - val_loss: 0.6039
Epoch 7/20
1188/1188 - 93s - 78ms/step - accuracy: 0.8243 - loss: 0.6017 - val_accuracy: 0.8498 - val_loss: 0.5280
Epoch 8/20
1188/1188 - 93s - 78ms/step - accuracy: 0.8386 - loss: 0.5506 - val_accuracy: 0.8491 - val_loss: 0.5396
Epoch 9/20
1188/1188 - 93s - 78ms/step - accuracy: 0.8511 - loss: 0.5034 - val_accuracy: 0.8587 - val_loss: 0.5338
Epoch 10/20
1188/1188 - 93s - 78ms/step - accuracy: 0.8608 - loss: 0.4678 - val_accuracy:

W0000 00:00:1710574687.159070     107 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


594/594 - 4s - 6ms/step - accuracy: 0.8775 - loss: 0.4547


W0000 00:00:1710574690.399744     107 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



Fold 5 - Test Accuracy: 87.75%
Trained model saved successfully at: /kaggle/working/DFNet_OpenWorld_WTFPAD.keras
